In [1]:
import simulate
import baseline
import dataload
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
#import pypots
from pypots.data import load_specific_dataset, mcar, masked_fill
from pypots.imputation import SAITS, BRITS
from pypots.utils.metrics import cal_mae, cal_rmse, cal_mre

c:\Users\RUSHI\anaconda3\envs\tf\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# PAMAPS2

In [2]:
dict = dataload.pamaps2(window_size=40, drop_nan=True)
X = dict["X"]
y = dict["y"]

In [3]:
# MCAR
dict_MCAR = simulate.simulate_nan(X, 0.1, "MCAR")
X_intact_MCAR = dict_MCAR['X_init']
X_missing_MCAR = dict_MCAR['X_incomp']
X_mask_MCAR = dict_MCAR['mask']

# MAR - logistic
dict_MAR = simulate.simulate_nan(X, 0.1, "MAR")
X_intact_MAR = dict_MAR['X_init']
X_missing_MAR = dict_MAR['X_incomp']
X_mask_MAR = dict_MAR['mask']

# MNAR - logistic
dict_MAR = simulate.simulate_nan(X, 0.1, "MNAR", opt="logistic")
X_intact_MNAR = dict_MAR['X_init']
X_missing_MNAR = dict_MAR['X_incomp']
X_mask_MNAR = dict_MAR['mask']

X_intact = [X_intact_MCAR, X_intact_MAR, X_intact_MNAR]
X_missing = [X_missing_MCAR, X_missing_MAR, X_missing_MNAR]
X_mask = [X_mask_MCAR, X_mask_MAR, X_mask_MNAR]
missingness = ["MCAR", "MAR", "MNAR"]

In [10]:
result_pamaps2_imputation = []

In [5]:
# Zero-filling imputation
res = []
for i in range(3):
    X_zero_imputed = baseline.zero_filling_imputation(X_missing[i])
    res.append(cal_mae(X_intact[i], X_zero_imputed, X_mask[i]))
    res.append(cal_rmse(X_intact[i], X_zero_imputed, X_mask[i]))
    res.append(cal_mre(X_intact[i], X_zero_imputed, X_mask[i]))

result_pamaps2_imputation.append(res)

In [8]:
# Mean imputation
res = []

for i in range(3):
    X_mean_imputed = baseline.mean_imputation(X_missing[i])
    res.append(cal_mae(X_intact[i], X_mean_imputed, X_mask[i]))
    res.append(cal_rmse(X_intact[i], X_mean_imputed, X_mask[i]))
    res.append(cal_mre(X_intact[i], X_mean_imputed, X_mask[i]))

result_pamaps2_imputation.append(res)

C:\Users\RUSHI\AppData\Local\Temp\ipykernel_31112\1594950506.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_pamaps2_imputation.append(res)


,MCAR mae,MCAR rmse,MCAR mre,MAR mae,MAR rmse,MAR mre,MNAR mae,MNAR rmse,MNAR mre,0
zero-filling imputation,43.165854,297.015161,1.274613e+18,60.875319,404.242705,8.970979e+17,51.925031,354.215814,1.535005e+18,NaN
mean imputation,43.165854,297.015161,1.274613e+18,60.875319,404.242705,8.970979e+17,51.925031,354.215814,1.535005e+18,NaN
median imputation,43.165854,297.015161,1.274613e+18,60.875319,404.242705,8.970979e+17,51.925031,354.215814,1.535005e+18,NaN
BRITS,43.165854,297.015161,1.274613e+18,60.875319,404.242705,8.970979e+17,51.925031,354.215814,1.535005e+18,NaN
SAITS,43.165854,297.015161,1.274613e+18,60.875319,404.242705,8.970979e+17,51.925031,354.215814,1.535005e+18,NaN
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1789.493249
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2112.168979
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.979808
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2688.961536
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2883.334233


In [9]:
# Median imputation
res = []

for i in range(3):
    X_median_imputed = baseline.median_imputation(X_missing[i])
    res.append(cal_mae(X_intact[i], X_median_imputed, X_mask[i]))
    res.append(cal_rmse(X_intact[i], X_median_imputed, X_mask[i]))
    res.append(cal_mre(X_intact[i], X_median_imputed, X_mask[i]))

result_pamaps2_imputation.append(res)

KeyboardInterrupt: 

In [11]:
# BRITS

res = []

for i in range(3):
    brits = BRITS(n_steps=40, n_features=52,rnn_hidden_size=64, epochs=20)
    brits.fit(X_missing[i])
    imputation = brits.impute(X_missing[i])
    res.append(cal_mae(imputation, X_intact[i], X_mask[i]))
    res.append(cal_rmse(imputation, X_intact[i], X_mask[i]))
    res.append(cal_mre(imputation, X_intact[i], X_mask[i]))

result_pamaps2_imputation.append(res)

Model initialized successfully. Number of the trainable parameters: 122528


RuntimeError: [enforce fail at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\c10\core\impl\alloc_cpu.cpp:81] data. DefaultCPUAllocator: not enough memory: you tried to allocate 2362164480 bytes.

In [ ]:
# SAITS

res = []

for i in range(3):
    saits = SAITS(n_steps=40, n_features=52, n_layers=2, d_model=256, d_inner=128, n_head=4, d_k=64, d_v=64, dropout=0.1, epochs=20)
    saits.fit(X_missing[i])
    imputation = saits.impute(X_missing[i])
    res.append(cal_mae(imputation, X_intact[i], X_mask[i]))
    res.append(cal_rmse(imputation, X_intact[i], X_mask[i]))
    res.append(cal_mre(imputation, X_intact[i], X_mask[i]))

result_pamaps2_imputation.append(res)

In [6]:
errors = ["mae", "rmse", "mre"]
cols = []

for i in missingness:
    for j in errors:
        cols.append(i+" "+j)

methods = ["zero-filling imputation", "mean imputation", "median imputation", "BRITS", "SAITS"]
result_pamaps2_imputation = pd.DataFrame(result_pamaps2_imputation, columns = cols, index=methods)

In [ ]:
result_pamaps2_imputation

,MCAR mae,MCAR rmse,MCAR mre,MAR mae,MAR rmse,MAR mre,MNAR mae,MNAR rmse,MNAR mre
zero-filling imputation,43.165854,297.015161,1.274613e+18,60.875319,404.242705,8.970979e+17,51.925031,354.215814,1.535005e+18
mean imputation,43.165854,297.015161,1.274613e+18,60.875319,404.242705,8.970979e+17,51.925031,354.215814,1.535005e+18
median imputation,43.165854,297.015161,1.274613e+18,60.875319,404.242705,8.970979e+17,51.925031,354.215814,1.535005e+18
BRITS,43.165854,297.015161,1.274613e+18,60.875319,404.242705,8.970979e+17,51.925031,354.215814,1.535005e+18
SAITS,43.165854,297.015161,1.274613e+18,60.875319,404.242705,8.970979e+17,51.925031,354.215814,1.535005e+18
